# DNN approach for maRAPID

Goodsol Lee, Netlab, Seoul National University

## Get datasets

In [134]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os

#configuration for gpu usage
conf = tf.ConfigProto()
# you can modify below as you want
#conf.gpu_options.per_process_gpu_memory_fraction = 0.4
#conf.gpu_options.allow_growth = True
os.environ['CUDA_VISIBLE_DEVICES']='0'
print(tf.__version__)

1.12.0


<font size="5">Sample training data, validation data, test data from raw data</font> 
data is classified with its cell id

In [135]:
node_num = 155
cell_num = 7
num_RSSI_sample = 5

data_set = np.empty((cell_num, num_RSSI_sample))
channel_label_set = np.empty((cell_num, 1))
TA_label_set = np.empty((cell_num, 1))

for node_index in range(1,node_num+1):
    print('Get sample from node'+str(node_index))
    file_name = 'data/node-'+str(node_index)+'.txt'
    previous_cellId = -1
    cell_sample = list()
    cell_sample_set = list()
    
    with open(file_name, 'r') as f:
        while 1:
            line = f.readline()
            if not line: break
            parsed_line =line.split(' ')
            
            rssi = parsed_line[9]
            channel_state = parsed_line[8]
            current_cellId = parsed_line[10]
            TA_region = parsed_line[11]
            
            if previous_cellId == current_cellId:
                cell_sample.append([rssi, channel_state, TA_region])
            else:
                previous_cellId = current_cellId
                if cell_sample:
                    cell_sample_set.append(cell_sample)
                    cell_sample = list()
        
        cell_sample_set.append(cell_sample)
        sample_length = 0
        for i in range(len(cell_sample_set)):
            sample_length += len(cell_sample_set[i])-num_RSSI_sample+1
        
        print(sample_length)
        
        temp_data_set = np.zeros((sample_length,num_RSSI_sample))
        temp_channel_label_set = np.zeros((sample_length,1))
        temp_TA_label_set = np.zeros((sample_length,1))
        
        for i in range(len(cell_sample_set)):
            cell_sample = cell_sample_set[i]
            cell_sample = np.array(cell_sample)
            for j in range(len(cell_sample_set[i])-num_RSSI_sample+1):
                temp_data_set[j] = cell_sample[j:j+num_RSSI_sample,0]
                temp_channel_label_set[j] = cell_sample[j+num_RSSI_sample-1,1]
                temp_TA_label_set[j] = cell_sample[j+num_RSSI_sample-1,2] 
              
    if node_index == 1:
        data_set = temp_data_set
        channel_label_set = temp_channel_label_set
        TA_label_set = temp_TA_label_set
    else:
        data_set = np.append(data_set,temp_data_set,axis = 0)
        channel_label_set = np.append(channel_label_set,temp_channel_label_set,axis = 0)
        TA_label_set = np.append(TA_label_set,temp_TA_label_set,axis=0)
    
print('Sampling is completed, sample length: ',data_set.shape[0])

Get sample from node1
25778
Get sample from node2
15593
Get sample from node3
25385
Get sample from node4
18592
Get sample from node5
26587
Get sample from node6
21198
Get sample from node7
30593
Get sample from node8
52985
Get sample from node9
26591
Get sample from node10
21804
Get sample from node11
21577
Get sample from node12
16782
Get sample from node13
33386
Get sample from node14
23393
Get sample from node15
52182
Get sample from node16
34788
Get sample from node17
26387
Get sample from node18
29190
Get sample from node19
22387
Get sample from node20
24587
Get sample from node21
41982
Get sample from node22
40972
Get sample from node23
11796
Get sample from node24
23987
Get sample from node25
27986
Get sample from node26
42386
Get sample from node27
29387
Get sample from node28
20792
Get sample from node29
24191
Get sample from node30
25194
Get sample from node31
35191
Get sample from node32
34187
Get sample from node33
25191
Get sample from node34
20586
Get sample from node35


In [136]:
for i in range(1):
    print('me')

me


In [137]:
import random

data_length = data_set.shape[0]
valid_num = 100000
test_num = 100000

#get test set
test_index = random.sample(range(0,data_length-test_num),test_num)
test_data = data_set[test_index]
test_channel_label = channel_label_set[test_index]
test_TA_label = TA_label_set[test_index]

#get training set/validation set
train_data_set = np.delete(data_set,test_index,axis=0)
train_channel_label_set = np.delete(channel_label_set,test_index,axis=0)
train_TA_label_set = np.delete(TA_label_set,test_index,axis=0)
data_length = train_data_set.shape[0]

valid_index = random.sample(range(0,data_length),valid_num)

training_data = train_data_set
training_channel_label = train_channel_label_set
training_TA_label = train_TA_label_set

valid_data = train_data_set[valid_index]
valid_channel_label = train_channel_label_set[valid_index]
valid_TA_label = train_TA_label_set[valid_index]

save_data ={
    'training_data':training_data,
    'training_channel_label':training_channel_label,
    'training_TA_label':training_TA_label,
    
    'valid_data':valid_data,
    'valid_channel_label':valid_channel_label,
    'valid_TA_label':valid_TA_label,
    
    'test_data':test_data,
    'test_channel_label':test_channel_label,
    'test_TA_label':test_TA_label
}

In [138]:
# get name of the data path
data_path = 'data/save_data'+str(num_RSSI_sample)+'.pickle'

In [139]:
# save data
with open(data_path,'wb') as f:
    pickle.dump(save_data,f,pickle.HIGHEST_PROTOCOL)

In [140]:
# restore data
with open(data_path,'rb') as f:
    save = pickle.load(f)
    
    train_dataset = save['training_data']
    train_channel_labels = save['training_channel_label']
    train_labels = save['training_TA_label']
    
    valid_dataset = save['valid_data']
    valid_channel_labels = save['valid_channel_label']
    valid_labels = save['valid_TA_label']
    
    test_dataset = save['test_data']
    test_channel_labels = save['test_channel_label']
    test_labels = save['test_TA_label']
    
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (3961943, 5) (3961943, 1)
Validation set (100000, 5) (100000, 1)
Test set (100000, 5) (100000, 1)


In [141]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, num_RSSI_sample)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    enc.fit(labels)
    labels = enc.transform(labels).toarray()
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Training set (3961943, 5) (3961943, 4)
Validation set (100000, 5) (100000, 4)
Test set (100000, 5) (100000, 4)


/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/gslee/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this wa

In [150]:
sample_size = num_RSSI_sample
num_labels = 4

layer1_output_num = 200
layer2_output_num = 200
layer3_output_num = 1024
layer4_output_num = 512
layer5_output_num = 256
num_steps = 1000
batch_size = 100000

graph_gs=tf.Graph()
with graph_gs.as_default():
    tf_dataset_gs=tf.placeholder(tf.float32, shape=(None, sample_size))
    tf_labels_gs=tf.placeholder(tf.float32, shape=(None, num_labels))
    keep_prob = tf.placeholder(tf.float32, shape=(None))
    
    lambda_reg = 0.1
        
    #Regularization
    regularizer = tf.contrib.layers.l2_regularizer(scale=lambda_reg)
    initializer = tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)
    #neural network consists of two lines
    dense1 = tf.layers.dense(tf_dataset_gs, layer1_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense1 = tf.nn.dropout(dense1, keep_prob = keep_prob)
    dense2 = tf.layers.dense(dense1,layer2_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense2 = tf.nn.dropout(dense2, keep_prob = keep_prob)
    dense3 = tf.layers.dense(dense2 ,layer3_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense3 = tf.nn.dropout(dense3, keep_prob = keep_prob)
    dense4 = tf.layers.dense(dense3 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense4 = tf.nn.dropout(dense4, keep_prob = keep_prob)
    dense5 = tf.layers.dense(dense4 ,layer4_output_num, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dense5 = tf.nn.dropout(dense5, keep_prob = keep_prob)
    logits_gs = tf.layers.dense(dense2, num_labels, activation=None)
    
    #Loss
    loss_gs = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_labels_gs, logits=logits_gs))
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.005
    
    decay_num =  50*(int)(train_labels.shape[0]/batch_size)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_num, 1, staircase=True)
    # Optimizer
    optimizer_gs = tf.train.AdamOptimizer(learning_rate).minimize(loss_gs, global_step=global_step)
    
    #Predictions for the training
    prediction_gs = tf.nn.softmax(logits_gs)

In [151]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.equal(np.argmax(predictions, 1), np.argmax(labels, 1)))
          / predictions.shape[0])

tf.reset_default_graph()
batch_num = (int)(train_labels.shape[0]/batch_size)

with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        random = np.random.permutation(train_labels.shape[0])
        loss = 0
        for b in range(batch_num):
            batch_data = train_dataset[random[b*batch_size:(b+1)*batch_size]]
            batch_labels = train_labels[random[b*batch_size:(b+1)*batch_size]].astype(float)
            feed_dict_gs = {tf_dataset_gs: batch_data, tf_labels_gs: batch_labels, keep_prob:1}
            _, l_gs, predictions_l = session_gs.run([optimizer_gs, loss_gs, prediction_gs], feed_dict=feed_dict_gs)
            loss += l_gs
        print("Minibatch loss at step %d: %f" % (step, loss))
        feed_dict_val_gs = {tf_dataset_gs: valid_dataset}
        valid_prediction_gs = session_gs.run(prediction_gs, feed_dict={tf_dataset_gs: valid_dataset, tf_labels_gs: valid_labels, keep_prob:1})
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction_gs, valid_labels))
        
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1.0}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))
    saver = tf.train.Saver()
    saver.save(session_gs, "./model_checkpoints/model1_sample5")

Initialized
Minibatch loss at step 0: 196.141833
Validation accuracy: 62.2%
Minibatch loss at step 1: 26.396725
Validation accuracy: 66.3%
Minibatch loss at step 2: 25.370799
Validation accuracy: 66.5%
Minibatch loss at step 3: 25.146505
Validation accuracy: 66.7%
Minibatch loss at step 4: 24.941774
Validation accuracy: 66.8%
Minibatch loss at step 5: 24.756759
Validation accuracy: 67.4%
Minibatch loss at step 6: 24.548635
Validation accuracy: 67.8%
Minibatch loss at step 7: 24.289489
Validation accuracy: 68.3%
Minibatch loss at step 8: 23.956219
Validation accuracy: 68.8%
Minibatch loss at step 9: 23.778859
Validation accuracy: 69.9%
Minibatch loss at step 10: 23.560466
Validation accuracy: 68.0%
Minibatch loss at step 11: 23.283591
Validation accuracy: 71.0%
Minibatch loss at step 12: 22.722817
Validation accuracy: 73.3%
Minibatch loss at step 13: 23.333528
Validation accuracy: 70.0%
Minibatch loss at step 14: 22.374476
Validation accuracy: 74.2%
Minibatch loss at step 15: 23.842502


Minibatch loss at step 128: 13.086385
Validation accuracy: 83.7%
Minibatch loss at step 129: 13.099674
Validation accuracy: 84.2%
Minibatch loss at step 130: 13.022986
Validation accuracy: 84.3%
Minibatch loss at step 131: 13.035093
Validation accuracy: 84.0%
Minibatch loss at step 132: 13.051832
Validation accuracy: 84.0%
Minibatch loss at step 133: 13.073709
Validation accuracy: 84.1%
Minibatch loss at step 134: 12.934206
Validation accuracy: 84.2%
Minibatch loss at step 135: 13.115965
Validation accuracy: 84.2%
Minibatch loss at step 136: 12.965169
Validation accuracy: 84.2%
Minibatch loss at step 137: 12.955444
Validation accuracy: 84.0%
Minibatch loss at step 138: 13.092422
Validation accuracy: 82.2%
Minibatch loss at step 139: 13.209695
Validation accuracy: 84.3%
Minibatch loss at step 140: 12.858247
Validation accuracy: 84.3%
Minibatch loss at step 141: 13.095658
Validation accuracy: 83.6%
Minibatch loss at step 142: 12.934825
Validation accuracy: 84.3%
Minibatch loss at step 14

Minibatch loss at step 255: 12.712940
Validation accuracy: 84.5%
Minibatch loss at step 256: 12.734427
Validation accuracy: 84.0%
Minibatch loss at step 257: 12.699126
Validation accuracy: 84.5%
Minibatch loss at step 258: 13.099239
Validation accuracy: 84.1%
Minibatch loss at step 259: 12.878498
Validation accuracy: 84.5%
Minibatch loss at step 260: 12.711273
Validation accuracy: 84.5%
Minibatch loss at step 261: 12.711323
Validation accuracy: 84.3%
Minibatch loss at step 262: 12.740034
Validation accuracy: 84.5%
Minibatch loss at step 263: 12.730412
Validation accuracy: 84.5%
Minibatch loss at step 264: 12.722443
Validation accuracy: 84.5%
Minibatch loss at step 265: 12.777644
Validation accuracy: 84.5%
Minibatch loss at step 266: 12.689363
Validation accuracy: 84.3%
Minibatch loss at step 267: 12.828300
Validation accuracy: 84.3%
Minibatch loss at step 268: 12.716781
Validation accuracy: 84.4%
Minibatch loss at step 269: 12.818011
Validation accuracy: 84.3%
Minibatch loss at step 27

Minibatch loss at step 382: 12.690132
Validation accuracy: 84.4%
Minibatch loss at step 383: 12.637375
Validation accuracy: 84.6%
Minibatch loss at step 384: 12.834776
Validation accuracy: 83.7%
Minibatch loss at step 385: 12.709634
Validation accuracy: 84.2%
Minibatch loss at step 386: 12.692164
Validation accuracy: 83.8%
Minibatch loss at step 387: 12.716841
Validation accuracy: 84.4%
Minibatch loss at step 388: 12.754663
Validation accuracy: 84.2%
Minibatch loss at step 389: 12.641999
Validation accuracy: 84.5%
Minibatch loss at step 390: 12.687772
Validation accuracy: 84.4%
Minibatch loss at step 391: 12.743832
Validation accuracy: 84.5%
Minibatch loss at step 392: 12.774736
Validation accuracy: 84.5%
Minibatch loss at step 393: 12.753732
Validation accuracy: 84.5%
Minibatch loss at step 394: 12.685497
Validation accuracy: 84.5%
Minibatch loss at step 395: 12.712541
Validation accuracy: 83.9%
Minibatch loss at step 396: 12.669353
Validation accuracy: 84.6%
Minibatch loss at step 39

Minibatch loss at step 509: 13.243498
Validation accuracy: 83.7%
Minibatch loss at step 510: 13.149188
Validation accuracy: 83.8%
Minibatch loss at step 511: 13.094628
Validation accuracy: 83.9%
Minibatch loss at step 512: 13.054545
Validation accuracy: 84.0%
Minibatch loss at step 513: 13.188131
Validation accuracy: 83.8%
Minibatch loss at step 514: 13.131030
Validation accuracy: 84.0%
Minibatch loss at step 515: 12.979955
Validation accuracy: 84.0%
Minibatch loss at step 516: 12.978640
Validation accuracy: 84.0%
Minibatch loss at step 517: 13.033070
Validation accuracy: 84.1%
Minibatch loss at step 518: 13.018911
Validation accuracy: 84.1%
Minibatch loss at step 519: 12.958427
Validation accuracy: 84.0%
Minibatch loss at step 520: 12.995307
Validation accuracy: 84.0%
Minibatch loss at step 521: 12.955455
Validation accuracy: 83.6%
Minibatch loss at step 522: 13.001669
Validation accuracy: 84.1%
Minibatch loss at step 523: 13.043022
Validation accuracy: 84.1%
Minibatch loss at step 52

Minibatch loss at step 636: 12.829516
Validation accuracy: 84.2%
Minibatch loss at step 637: 12.804310
Validation accuracy: 84.3%
Minibatch loss at step 638: 12.832629
Validation accuracy: 84.2%
Minibatch loss at step 639: 12.848453
Validation accuracy: 84.2%
Minibatch loss at step 640: 12.826543
Validation accuracy: 84.3%
Minibatch loss at step 641: 12.831778
Validation accuracy: 84.1%
Minibatch loss at step 642: 12.891431
Validation accuracy: 83.9%
Minibatch loss at step 643: 12.889243
Validation accuracy: 83.9%
Minibatch loss at step 644: 12.859484
Validation accuracy: 84.3%
Minibatch loss at step 645: 12.897097
Validation accuracy: 84.3%
Minibatch loss at step 646: 12.839060
Validation accuracy: 84.2%
Minibatch loss at step 647: 12.839496
Validation accuracy: 84.3%
Minibatch loss at step 648: 12.841994
Validation accuracy: 83.9%
Minibatch loss at step 649: 12.877537
Validation accuracy: 84.3%
Minibatch loss at step 650: 12.852992
Validation accuracy: 84.2%
Minibatch loss at step 65

Minibatch loss at step 763: 12.874909
Validation accuracy: 84.3%
Minibatch loss at step 764: 12.777615
Validation accuracy: 84.3%
Minibatch loss at step 765: 12.777847
Validation accuracy: 84.3%
Minibatch loss at step 766: 12.780188
Validation accuracy: 84.3%
Minibatch loss at step 767: 12.776856
Validation accuracy: 84.3%
Minibatch loss at step 768: 12.825111
Validation accuracy: 83.3%
Minibatch loss at step 769: 12.865451
Validation accuracy: 84.4%
Minibatch loss at step 770: 12.800143
Validation accuracy: 84.4%
Minibatch loss at step 771: 12.788373
Validation accuracy: 83.9%
Minibatch loss at step 772: 12.795536
Validation accuracy: 84.3%
Minibatch loss at step 773: 12.756991
Validation accuracy: 84.4%
Minibatch loss at step 774: 12.839328
Validation accuracy: 84.3%
Minibatch loss at step 775: 12.755700
Validation accuracy: 84.4%
Minibatch loss at step 776: 12.815478
Validation accuracy: 84.4%
Minibatch loss at step 777: 12.786743
Validation accuracy: 83.9%
Minibatch loss at step 77

Minibatch loss at step 890: 12.674675
Validation accuracy: 84.4%
Minibatch loss at step 891: 12.761278
Validation accuracy: 84.4%
Minibatch loss at step 892: 12.736193
Validation accuracy: 84.2%
Minibatch loss at step 893: 12.713186
Validation accuracy: 84.4%
Minibatch loss at step 894: 12.727803
Validation accuracy: 84.5%
Minibatch loss at step 895: 12.706827
Validation accuracy: 83.9%
Minibatch loss at step 896: 12.799183
Validation accuracy: 84.3%
Minibatch loss at step 897: 12.677703
Validation accuracy: 84.3%
Minibatch loss at step 898: 12.734668
Validation accuracy: 84.5%
Minibatch loss at step 899: 12.762061
Validation accuracy: 84.3%
Minibatch loss at step 900: 12.772408
Validation accuracy: 84.5%
Minibatch loss at step 901: 12.744033
Validation accuracy: 84.4%
Minibatch loss at step 902: 12.672641
Validation accuracy: 84.5%
Minibatch loss at step 903: 12.730224
Validation accuracy: 84.5%
Minibatch loss at step 904: 12.735980
Validation accuracy: 84.4%
Minibatch loss at step 90

In [93]:
with tf.Session(graph=graph_gs, config=conf) as session_gs:
    tf.global_variables_initializer().run()
    feed_dict_test_gs = {tf_dataset_gs: test_dataset, keep_prob:1}
    test_prediction_gs = session_gs.run(prediction_gs, feed_dict=feed_dict_test_gs)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction_gs, test_labels))

Test accuracy: 52.5%
